In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import gzip
from subprocess import check_output
import math
%matplotlib inline

In [ ]:
production_df = pd.read_excel('../data/tmdb_production_data.xlsx', index_col=0)
movie_df = pd.read_excel('../data/tmdb_movie_data.xlsx', index_col=0)
genre_df = pd.read_excel('../data/movie_genres_data.xlsx', index_col=0)

In [ ]:
genre_list = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

In [ ]:
revenue = production_df[['movie_title', 'movie_revenue', 'movie_budget', 'movie_profits']]
revenue = revenue[revenue['movie_revenue'] != 0]
revenue = revenue[revenue['movie_budget'] != 0]
revenue = revenue.drop(columns={'movie_revenue', 'movie_budget'})
revenue

In [ ]:
movie_ids = movie_df[['movie_id', 'movie_title']]
movie_ids

In [ ]:
revenue_ids = revenue.merge(movie_ids)
revenue_ids = revenue_ids[revenue_ids['movie_profits'] != 0]
revenue_ids = revenue_ids.drop(columns={'movie_title'})
revenue_ids

In [ ]:
genre_revenue = revenue_ids.merge(genre_df)
genre_revenue = genre_revenue.drop(columns={'movie_id', 'genre'})
genre_revenue['movie_profits'] = round(genre_revenue['movie_profits']/1000000, 0)
genre_revenue

In [ ]:
genre_revenue_data = []
for x in genre_list:
    genre_revenue_data.append(round(genre_revenue.loc[genre_revenue[x] == 1, 'movie_profits'].mean(),0))

In [ ]:
genre_revenue = pd.DataFrame(list(zip(genre_list, genre_revenue_data)),
               columns =['Genres', 'Average Profits (Millions)'])
genre_revenue = genre_revenue.sort_values(by=['Average Profits (Millions)'], ascending=True)
genre_revenue = genre_revenue[genre_revenue['Average Profits (Millions)'] > 100]

In [ ]:
means = list(genre_revenue['Average Profits (Millions)'])
means = [int(i) for i in means]
means

In [ ]:
plt.figure(figsize=(14,11))
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")

palette = {'Comedy': "#f7b23b", 'Animation':"#2a6671", 'Family':'#c05236', 
           'Adventure':'#a82200', 'Fantasy':'#3e6200', 'Action':'#391513', 'Science Fiction':'#391513'}

bar = sns.barplot(x='Genres', y='Average Profits (Millions)', data=genre_revenue, palette=palette, ci=False);

for x in range(len(means)):
    bar.text(x,means[x]+2.5, str(round(means[x],0)),
            fontdict = dict(color='white', fontsize=18),
            horizontalalignment = 'center')